In [1]:
import numpy as np
import pandas as pd 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import torch as to
import torch.nn as nn 
import torch.nn.functional as func
from torch.utils import data
from torch.utils.data import DataLoader
import random 
import time 
import torch.nn.utils.rnn as rnnutils
from sklearn.model_selection import train_test_split
import torchviz
from torchsummary import summary
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score 
from itertools import tee
import hashlib
from torch.optim.lr_scheduler import StepLR
import gc
import os 

Using TensorFlow backend.



### Info 
This notebook is a continuation of the 'baseline_take2_twin.ipynb' nootebook. 

### Setup 
Here we are working with bert inputs and one layer attention.

In [10]:
mbti_labels_1 = ['introverted', 'intuitive']
mbti_labels_2 = ['thinking', 'perceiving']

#Before each run 
RUN_IDENTIFICATOR = 'introverted_run4'
mbti_labels = ['introverted']

In [11]:
#CELL METADATA
DATASET = '/home/mbosnjak/Datasets/embedded_comments_last_100_min_20com_per_auth_w_wc_10_200_no_mbti.csv'
FOLDS = '/home/mbosnjak/Datasets/comments_last_100_min_20com_per_auth_w_wc_10_200.csv.folds.csv'
AUTHORS = '/home/mbosnjak/Datasets/author_profiles.csv'

MODELS_NAME = 'baseline2'
OUTPUT_DIR = f'mpr_{MODELS_NAME}/'
RESULTS = OUTPUT_DIR + f'results_{MODELS_NAME}_{RUN_IDENTIFICATOR}.csv'

RELEVANT_DIFFERENCE = 0.0001
NUMBER_OF_CLASSES = 2
STATE = 156
no_of_folds = 1
cuda_device_index = 1

MODELS_PREFORMANCE_COLUMNS = ['val_f1', 'val_precision_0', 'val_precision_1', 'val_precision_macro' ,\
                              'val_recall_0', 'val_recall_1', 'val_recall_macro', \
                              'test_f1', 'test_precision_0', 'test_precision_1', 'test_precision_macro',\
                              'test_recall_0', 'test_recall_1', 'test_recall_macro',\
                              'epoch']
MODELS_PARAMETER_COLUMNS = ['mbti_trait', 'fold', 'learning_rate', 'batch_size',\
                            'hidden_layer1', 'decay_rate', 'decay_epoch', 'run_identificator']
MODELS_META_DATA_COLUMNS = ['models_name', 'hash_id']

to.manual_seed(STATE)

use_GPU = True
cuda_device = to.device(cuda_device_index) if use_GPU else None
validation_p = .2
print_status_batch = 100
max_constant_f1 = 4

In [12]:
#CELL PARAMETERS
batch_sizes = [64]
learning_rates = [10**-5, 10**-6, 5*10**-6, 10**-7]
kernel_sizes = [12]
sentences_counts = [4]
hidden_layer1_sizes = [6, 8, 12]
n_epochs = 100
STEP_SIZE = 60
GAMMA = 0.5
#TODO: this is sentences per author not comments per author, quick fix is max size 200*100 (200 max sentences by comment * 100 comments)
comments_per_author = 20000
no_of_folds = 1

In [13]:
if(not os.path.isdir(OUTPUT_DIR)):
    os.makedirs(OUTPUT_DIR)

In [16]:
def get_checkpoints_name(models_identifier):
    return OUTPUT_DIR+"save_" + models_identifier + f'_{RUN_IDENTIFICATOR}_' + ".pt"

def get_predictions_file_name(models_identifier, target, fold):
    return OUTPUT_DIR + f'{target}_{fold}_{RUN_IDENTIFICATOR}_{models_identifier}_predictions'

def delimiter():
    print('-'*23)

In [5]:
#input_df = pd.read_csv(DATASET)
input_df = pd.read_csv(DATASET, nrows = 2000) # TODO: remove nrows
folds_df = pd.read_csv(FOLDS, usecols =['author', 'fold'])
authors_profiles_df = pd.read_csv(AUTHORS)
authors_with_mbti = list(authors_profiles_df[authors_profiles_df.introverted.notnull()]['author'])

In [6]:
#TODO: add comment delimiter -> to recognize comments end and beginning
def merger(comments):
    npad = [(0, max(sentences_counts)), (0, 0)]
    carpet = comments[:comments_per_author].as_matrix(columns=comments.columns[1:1025])
    carpet_padded = np.pad(carpet, pad_width=npad, mode='constant', constant_values=0)
    #print(f'before {carpet.shape} - after {carpet_padded.shape}')
    return to.tensor(carpet_padded)

In [7]:
#Data preparation
data_df = input_df[input_df['author'].isin(authors_with_mbti)]
del input_df
present_authors = sorted(data_df['author'].unique())
authors_indices = dict(zip(present_authors , list(range(len(present_authors)))))
input_x_df = data_df.sort_values(by=['author', 'Unnamed: 0']).groupby(['author']).apply(merger).tolist()
del data_df

/home/mbosnjak/.conda/envs/pero/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [34]:
results = pd.DataFrame(data=None, columns=MODELS_META_DATA_COLUMNS+MODELS_PARAMETER_COLUMNS+MODELS_PREFORMANCE_COLUMNS)
for name in MODELS_PREFORMANCE_COLUMNS:
    results = results.astype({name: 'float64'})
for name in MODELS_META_DATA_COLUMNS:
    results = results.astype({name: 'object'})

In [42]:
gc.collect()

613

In [20]:
def set_hash(index):
    global results
    identifier = hashlib.md5(''.join([str(x) for x in results[MODELS_PARAMETER_COLUMNS].iloc[index]]).encode('utf-8')).hexdigest()
    results.at[index, 'hash_id'] = identifier
    return identifier

def create_new_model_input(mbti_trait, fold, learning_rate, batch_size,\
                           kernels_count, sentences_count, hidden):
    global results
    results = results.append({'models_name':MODELS_NAME, 'mbti_trait' : mbti_trait,\
                              'fold' : fold, 'learning_rate' : learning_rate,\
                              'batch_size' : batch_size, 'hidden_layer1' : hidden, 'decay_rate' : GAMMA,\
                              'decay_epoch' : STEP_SIZE, 'run_identificator' : RUN_IDENTIFICATOR,\
                             'hash_id': 'DUMMY_VALUE'}, ignore_index=True)
    return set_hash(len(results)-1)

def update_models_val_results(models_identifier, val_f1, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_recall_1, val_recall_m, epoch):
    global results
    results.loc[results['hash_id'] == models_identifier, 'val_f1'] = val_f1
    results.loc[results['hash_id'] == models_identifier, 'val_precision_0'] = val_precision_0
    results.loc[results['hash_id'] == models_identifier, 'val_precision_1'] = val_precision_1
    results.loc[results['hash_id'] == models_identifier, 'val_precision_macro'] = val_precision_m
    results.loc[results['hash_id'] == models_identifier, 'val_recall_0'] = val_recall_0
    results.loc[results['hash_id'] == models_identifier, 'val_recall_1'] = val_recall_1
    results.loc[results['hash_id'] == models_identifier, 'val_recall_macro'] = val_recall_m
    results.loc[results['hash_id'] == models_identifier, 'epoch'] = epoch
    
def update_models_test_results(models_identifier, test_f1, test_precision_0, test_precision_1, precision_m, test_recall_0, test_recall_1, test_recall_m):
    global results
    results.loc[results['hash_id'] == models_identifier, 'test_f1'] = test_f1
    results.loc[results['hash_id'] == models_identifier, 'test_precision_0'] = test_precision_0
    results.loc[results['hash_id'] == models_identifier, 'test_precision_1'] = test_precision_1
    results.loc[results['hash_id'] == models_identifier, 'test_precision_macro'] = test_precision_m
    results.loc[results['hash_id'] == models_identifier, 'test_recall_0'] = test_recall_0
    results.loc[results['hash_id'] == models_identifier, 'test_recall_1'] = test_recall_1
    results.loc[results['hash_id'] == models_identifier, 'test_recall_macro'] = test_recall_m
    
def get_best_models_data(target, fold):
    return results.loc[results[(results.mbti_trait == target) & (results.fold == fold)]['val_f1'].idxmax()]
    

In [10]:
create_new_model_input('dfsfdsf',333,3,223333,1,13)
results.dtypes

models_name              object
hash_id                  object
mbti_trait               object
fold                     object
learning_rate            object
batch_size               object
kernels_count            object
sentences_count          object
decay_rate              float64
decay_epoch              object
run_identificator        object
val_f1                  float64
val_precision_0         float64
val_precision_1         float64
val_precision_macro     float64
val_recall_0            float64
val_recall_1            float64
val_recall_macro        float64
test_f1                 float64
test_precision_0        float64
test_precision_1        float64
test_precision_macro    float64
test_recall_0           float64
test_recall_1           float64
test_recall_macro       float64
epoch                   float64
dtype: object

In [18]:
results

,models_name,hash_id,mbti_trait,fold,learning_rate,batch_size,kernels_count,sentences_count,decay_rate,decay_epoch,...,val_recall_1,val_recall_macro,test_f1,test_precision_0,test_precision_1,test_precision_macro,test_recall_0,test_recall_1,test_recall_macro,epoch


### FLOW

In [21]:
def get_input_output(data_authors, target):
    output_prep = authors_profiles_df[authors_profiles_df['author'].isin(data_authors)][target].tolist()
    input_indices = [index for author, index in authors_indices.items() if author in data_authors]
    return input_indices, output_prep

def balance_binary_data(input_df, output_df):
    positive_indices = [index for index, element in enumerate(output_df) if element == 1.0]
    negative_indices = [index for index, element in enumerate(output_df) if element == 0.0]
    negative_count = len(negative_indices)
    positive_count = len(positive_indices)
    
    if(positive_count > negative_count):
        return create_balanced_data(positive_indices, negative_indices, input_df, output_df)
    elif(negative_count > positive_count):
        return create_balanced_data(negative_indices, positive_indices, input_df, output_df)
        
def create_balanced_data(more_frequent, less_frequent, input_df, output_df):
    more_frequent_count = len(more_frequent)
    less_frequent_count = len(less_frequent)
    constant_multiplyer = more_frequent_count // less_frequent_count
    remaining_additions = more_frequent_count % less_frequent_count
    balanced_indices = less_frequent*constant_multiplyer + less_frequent[:remaining_additions] + more_frequent
    output_df_balanced = [output_df[index] for index in balanced_indices]
    input_df_balanced = [input_df[index] for index in balanced_indices]
    return input_df_balanced, output_df_balanced
    
def data_preparation(target, fold):
    test_data_authors = folds_df[(folds_df['fold'] == fold) & (folds_df['author'].isin(present_authors))]['author'].tolist()
    train_data_authors = folds_df[(folds_df['fold'] != fold) & (folds_df['author'].isin(present_authors))]['author'].tolist()
    train_input_indices, train_output = get_input_output(train_data_authors, target)
    test_input_indices, test_output = get_input_output(test_data_authors, target)

    train_input_indices, val_input_indices, train_output, val_output = train_test_split(train_input_indices, train_output, test_size=validation_p, random_state=STATE)
    assert len(train_input_indices) == len(train_output)
    assert len(test_input_indices) == len(test_output)
    assert len(val_input_indices) == len(val_output)
    return train_input_indices, train_output, val_input_indices, val_output, test_input_indices, test_output

def chunks(l, n):
    return [l[i:i + n] for i in range(0, len(l), n)]  

def create_minibatches(data_X, data_y, minibatch_size, cuda_dev):
    for idx_list in chunks(range(len(data_X)), minibatch_size):
        data_X_indices = [data_X[index] for index in idx_list]
        data_y_idx = [int(data_y[index]) for index in idx_list]
        
        minibatch_X = [input_x_df[index] for index in data_X_indices]
        minibatch_X = rnnutils.pad_sequence(minibatch_X, batch_first=True, padding_value = 0) 
        minibatch_X = minibatch_X.unsqueeze(1)
        minibatch_y = to.tensor(data_y_idx)
        if cuda_dev is not None:
            minibatch_X = minibatch_X#.to(device=cuda_dev, dtype=to.float)
            minibatch_y = minibatch_y.to(device=cuda_dev, dtype=to.long)

        yield((minibatch_X, minibatch_y))   


In [61]:
class AttentiveCarpet(nn.Module):
    def __init__ (self, embedding_size = 1024):
        super(AttentiveCarpet, self).__init__()
        self.encoder_attentive_pooling_projection = nn.Linear(embedding_size, 1)
        self.linear = to.Linear()
        self.softmax = nn.Softmax(dim=1)
    
        
    def forward(self, input_batch):
        self_attentive_logits = self.encoder_attentive_pooling_projection(input_batch).squeeze(2)
        self_weights = util.softmax(self_attentive_logits)
        self_attentive_pool = util.weighted_sum(input_batch, self_weights)
        
        return self_attentive_pool


In [36]:
def calcMeasures(total_logits, total_true, threshold):
    total_preds = []
    for minibatch_logits in total_logits:
        minibatch_preds = minibatch_logits[:,1] > threshold
        total_preds.append(minibatch_preds)
    total_preds = to.cat(total_preds)
    total_true = to.cat(total_true)
    F1 = f1_score(total_true.cpu(), total_preds.cpu(), pos_label = 1, average='macro')
    precision_m = precision_score(total_true.cpu(), total_preds.cpu(), average='macro')
    precision_1 = precision_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    precision_0 = precision_score(total_true.cpu(), total_preds.cpu(), pos_label = 0)
    recall_m = recall_score(total_true.cpu(), total_preds.cpu(), average='macro')
    recall_1 = recall_score(total_true.cpu(), total_preds.cpu(), pos_label = 1)
    recall_0 = recall_score(total_true.cpu(), total_preds.cpu(), pos_label = 0)
    return F1, precision_0, precision_1, precision_m, recall_0, recall_1, recall_m

def apply_model(model, loss_func, data_loader):
    model.eval()
    total_loss = 0
    predicted = []
    true_output = []
    minibatch_loss = 0
    with to.no_grad():
        n_batch = 0
        for inputs, labels in data_loader:
            n_batch += 1
            minibatch_logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss_func(minibatch_logits, labels)
            
            total_loss += minibatch_loss
            predicted.append(minibatch_logits)
            true_output.append(labels)
            
    return total_loss, predicted, true_output

def optimize_models_params(trait, fold, train_indices, output, val_input_indices, val_output):
    print('Start training ...')
    
    all_sum = len(output)
    positive = sum(output)
    negative = all_sum - positive
    print(f'Postive: {positive}: Negative: {negative}')
    loss_weights = to.tensor([negative, positive])
    loss_weights = loss_weights.to(device=cuda_device, dtype=to.float)
    loss = nn.CrossEntropyLoss(weight=loss_weights)
    
    train_input_indices, train_output = balance_binary_data(train_indices, output)
    
    for batch_size in batch_sizes:
        for kernel_size in kernel_sizes:
            for sent_size in sentences_counts:
                for hidden in hidden_layer1_sizes:
                    for learning_rate in learning_rates:                

                        train_loader = create_minibatches(train_input_indices, train_output, batch_size, cuda_device)
                        val_loader = create_minibatches(val_input_indices, val_output, batch_size, cuda_device)

                        delimiter()
                        delimiter()
                        print(f'Starting training for BS: {batch_size} LR: {learning_rate}')
                        delimiter()

                        models_identifier = create_new_model_input(trait, fold, learning_rate, batch_size, kernel_size, sent_size, hidden)

                        model = AttentiveCarpet(kernels_count=kernel_size, sentences_count=sent_size, hidden1=hidden)
                        if use_GPU:
                              model.to(cuda_device)

                        optimizer = to.optim.Adam(model.parameters(), learning_rate, amsgrad = True)

                        exp_lr_scheduler = StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

                        train_model(train_loader, val_loader, models_identifier, model, loss, optimizer, exp_lr_scheduler)
                        results.to_csv(RESULTS)


def train_model(train_loader, val_loader, models_identifier, model, loss, optimizer, exp_lr_scheduler):
    best_f1_val = 0
    last_f1 = 0
    counter = 0
    
    for epoch in range(0, n_epochs):
        epoch_start = time.time()
        model.train()
        total_loss = 0
        n_batch = 0
        
        predicted = []
        true_output = []
        
        train_loader, train_loader_backup = tee(train_loader)
        for inputs, labels in train_loader_backup:

            n_batch += 1
            #print(f"Current batch is {n_batch}")

            model.zero_grad()

            logits = model(inputs.to(device=cuda_device, dtype=to.float))
            minibatch_loss = loss(logits, labels)
            total_loss += minibatch_loss
            minibatch_loss.backward()
            optimizer.step()
            predicted.append(logits)
            true_output.append(labels)
            
            if(n_batch % print_status_batch == 0):
                
                delimiter()               
                #preds = logits[:,1] > 0.5
                #train_f1_score = f1_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy(), pos_label = 1, average='macro')
                print(f"Train - Epoch {epoch} - batch {n_batch}, batch loss is {minibatch_loss:.6f}")
                delimiter()

        
        
        delimiter()
        exp_lr_scheduler.step()
        val_loader, val_loader_backup = tee(val_loader)
        val_loss, val_logits, true_val = apply_model(model, loss, val_loader_backup)
        val_f1_score, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_recall_1, val_recall_m = calcMeasures(val_logits, true_val, 0.5)
        train_f1_score, _, _, _, _, _,_ = calcMeasures(predicted, true_output, 0.5)
        
        print(f'Epoch {epoch} end: {time.time()-epoch_start}, F1 is: {train_f1_score}')
        print(f'Validation loss: {val_loss:.7f} - F1 score: {val_f1_score:.7f}')
        print(f'0 class -> precision: {val_precision_0:.7f} - recall: {val_recall_0:.7f}')
        print(f'1 class -> precision: {val_precision_1:.7f} - recall: {val_recall_1:.7f}')
        print(f'precision: {val_precision_m:.7f} - recall: {val_recall_m:.7f} - MACRO')
        delimiter()
        
        if (abs(val_f1_score - last_f1) <= RELEVANT_DIFFERENCE):
            counter += 1
        else:
            counter = 0
        
        last_f1 = val_f1_score
        
        if(counter >= max_constant_f1):
            return
        
        if (val_f1_score > best_f1_val):
            update_models_val_results(models_identifier, val_f1_score, val_precision_0, val_precision_1, val_precision_m, val_recall_0, val_precision_1, val_recall_m, epoch)
            best_f1_val = val_f1_score
        # save a checkpoint
            best_checkpoint_filename = get_checkpoints_name(models_identifier)
            to.save({
              "models_identifier" : models_identifier, 
              "model_state_dict" : model.state_dict(),
              "optimizer_state_dict" : optimizer.state_dict()
            }, best_checkpoint_filename)
    

In [ ]:
for target in mbti_labels:
    print(f"Creating model for target: {target}")
    for fold in range(0, no_of_folds):
        
        print(f'Processing fold {fold}...')
        
        train_input_indices, train_output, val_input_indices, val_output, test_input_indices, test_output = data_preparation(target, fold)
        
        optimize_models_params(target, fold, train_input_indices, train_output, val_input_indices, val_output)
        
        
        best_models = get_best_models_data(target, fold)
        #Recreate model
        models_identifier = best_models['hash_id']
        model = AttentiveCarpet(kernels_count=best_models['kernels_count'], sentences_count=best_models['sentences_count'], hidden1=best_models['hidden_layer1'])
        if use_GPU:
              model.to(cuda_device)

        checkpoint = to.load(get_checkpoints_name(models_identifier))
        model.load_state_dict(checkpoint["model_state_dict"])
        loss = nn.CrossEntropyLoss()

        print("."*15)
        print("."*15)
        
        print(f'Apply best model to test for {target} on fold {fold}.')
        test_loader = create_minibatches(test_input_indices, test_output, 1, cuda_device)
        test_loss, test_logits, test_true = apply_model(model, loss, test_loader)
        test_f1_score, test_precision_0, test_precision_1, test_precision_m, test_recall_0, test_recall_1, test_recall_m = calcMeasures(test_logits, test_true, 0.5)
        print(f'Test loss: {test_loss:.5f} - F1 score: {test_f1_score:.7f} ')
        print(f'0 class -> precision: {test_precision_0:.7f} - recall: {test_recall_0:.7f}')
        print(f'1 class -> precision: {test_precision_1:.7f} - recall: {test_recall_1:.7f}')
        print(f'precision: {test_precision_m:.7f} - recall: {test_recall_m:.7f} - MACRO')
              
        update_models_test_results(models_identifier, test_f1_score, test_precision_0, test_precision_1, test_precision_m, test_recall_0, test_recall_1, test_recall_m)
        with open(get_predictions_file_name(models_identifier, target, fold), 'w+') as f:
            for pre_l, true_l in zip(test_logits, test_true):
                f.write(f'{pre_l.cpu()[0][0]}, {pre_l.cpu()[0][1]}, {true_l.cpu().tolist()[0]}\n')
        results.to_csv(RESULTS)
        
        ##Save best models data
        
        print(f"+++ Finished with training and testing model for {target} on fold {fold}. +++")
        
        print("."*15)
        print("."*15)

Creating model for target: introverted
Processing fold 0...
Start training ...
Postive: 4433.0: Negative: 1219.0
-----------------------
-----------------------
Starting training for BS: 64 KS: 12 SS: 4 LR: 1e-05 HL: 6
-----------------------
-----------------------
Train - Epoch 0 - batch 100, batch loss is 0.693147
-----------------------
-----------------------
Epoch 0 end: 9.461567878723145, F1 is: 0.3333333333333333
Validation loss: 15.9423885 - F1 score: 0.1750292
0 class -> precision: 0.2121641 - recall: 1.0000000
1 class -> precision: 0.0000000 - recall: 0.0000000
precision: 0.1060820 - recall: 0.5000000 - MACRO
-----------------------
-----------------------
Train - Epoch 1 - batch 100, batch loss is 0.693147
-----------------------
-----------------------
Epoch 1 end: 4.733005046844482, F1 is: 0.3333333333333333
Validation loss: 15.9423885 - F1 score: 0.1750292
0 class -> precision: 0.2121641 - recall: 1.0000000
1 class -> precision: 0.0000000 - recall: 0.0000000
precision: 0

-----------------------
Train - Epoch 4 - batch 100, batch loss is 0.677154
-----------------------
-----------------------
Epoch 4 end: 4.879780054092407, F1 is: 0.3915644201973074
Validation loss: 15.6099758 - F1 score: 0.4808864
0 class -> precision: 0.1969697 - recall: 0.0866667
1 class -> precision: 0.7862715 - recall: 0.9048474
precision: 0.4916206 - recall: 0.4957570 - MACRO
-----------------------
-----------------------
Train - Epoch 5 - batch 100, batch loss is 0.677172
-----------------------
-----------------------
Epoch 5 end: 5.135977745056152, F1 is: 0.39218016100405606
Validation loss: 15.6103735 - F1 score: 0.4808864
0 class -> precision: 0.1969697 - recall: 0.0866667
1 class -> precision: 0.7862715 - recall: 0.9048474
precision: 0.4916206 - recall: 0.4957570 - MACRO
-----------------------
-----------------------
Train - Epoch 6 - batch 100, batch loss is 0.677191
-----------------------
-----------------------
Epoch 6 end: 4.466697692871094, F1 is: 0.3923852775711637

-----------------------
Train - Epoch 10 - batch 100, batch loss is 0.699872
-----------------------
-----------------------
Epoch 10 end: 6.330445051193237, F1 is: 0.4200898754295954
Validation loss: 16.0149460 - F1 score: 0.3047805
0 class -> precision: 0.2060345 - recall: 0.7966667
1 class -> precision: 0.7598425 - recall: 0.1732496
precision: 0.4829385 - recall: 0.4849581 - MACRO
-----------------------
-----------------------
Train - Epoch 11 - batch 100, batch loss is 0.699774
-----------------------
-----------------------
Epoch 11 end: 6.542770624160767, F1 is: 0.42305151688332715
Validation loss: 16.0132980 - F1 score: 0.3114097
0 class -> precision: 0.2076455 - recall: 0.7966667
1 class -> precision: 0.7680608 - recall: 0.1813285
precision: 0.4878532 - recall: 0.4889976 - MACRO
-----------------------
-----------------------
Train - Epoch 12 - batch 100, batch loss is 0.699669
-----------------------
-----------------------
Epoch 12 end: 6.221776485443115, F1 is: 0.4251698286

-----------------------
Train - Epoch 30 - batch 100, batch loss is 0.698566
-----------------------
-----------------------
Epoch 30 end: 5.309837579727173, F1 is: 0.47363207768422844
Validation loss: 15.9939375 - F1 score: 0.3499372
0 class -> precision: 0.2047847 - recall: 0.7133333
1 class -> precision: 0.7669377 - recall: 0.2540395
precision: 0.4858612 - recall: 0.4836864 - MACRO
-----------------------
-----------------------
Train - Epoch 31 - batch 100, batch loss is 0.698539
-----------------------
-----------------------
Epoch 31 end: 5.436961650848389, F1 is: 0.4762622352022349
Validation loss: 15.9935350 - F1 score: 0.3512639
0 class -> precision: 0.2051774 - recall: 0.7133333
1 class -> precision: 0.7681941 - recall: 0.2558348
precision: 0.4866857 - recall: 0.4845841 - MACRO
-----------------------
-----------------------
Train - Epoch 32 - batch 100, batch loss is 0.698516
-----------------------
-----------------------
Epoch 32 end: 5.853017330169678, F1 is: 0.4794220534

-----------------------
Train - Epoch 50 - batch 100, batch loss is 0.698202
-----------------------
-----------------------
Epoch 50 end: 6.032998561859131, F1 is: 0.5119765269422878
Validation loss: 15.9884434 - F1 score: 0.3747297
0 class -> precision: 0.2077001 - recall: 0.6833333
1 class -> precision: 0.7775176 - recall: 0.2980251
precision: 0.4926088 - recall: 0.4906792 - MACRO
-----------------------
-----------------------
Train - Epoch 51 - batch 100, batch loss is 0.698192
-----------------------
-----------------------
Epoch 51 end: 6.415438652038574, F1 is: 0.5121474089888187
Validation loss: 15.9884224 - F1 score: 0.3747297
0 class -> precision: 0.2077001 - recall: 0.6833333
1 class -> precision: 0.7775176 - recall: 0.2980251
precision: 0.4926088 - recall: 0.4906792 - MACRO
-----------------------
-----------------------
Train - Epoch 52 - batch 100, batch loss is 0.698181
-----------------------
-----------------------
Epoch 52 end: 6.523768186569214, F1 is: 0.51325035738

...

-----------------------
Train - Epoch 2 - batch 100, batch loss is 0.811244
-----------------------
-----------------------
Epoch 2 end: 5.24880051612854, F1 is: 0.3333333333333333
Validation loss: 17.5635700 - F1 score: 0.1750292
0 class -> precision: 0.2121641 - recall: 1.0000000
1 class -> precision: 0.0000000 - recall: 0.0000000
precision: 0.1060820 - recall: 0.5000000 - MACRO
-----------------------
-----------------------
Train - Epoch 3 - batch 100, batch loss is 0.801674
-----------------------
-----------------------
Epoch 3 end: 4.759392023086548, F1 is: 0.3333333333333333
Validation loss: 17.4916935 - F1 score: 0.1750292
0 class -> precision: 0.2121641 - recall: 1.0000000
1 class -> precision: 0.0000000 - recall: 0.0000000
precision: 0.1060820 - recall: 0.5000000 - MACRO
-----------------------
-----------------------
Train - Epoch 4 - batch 100, batch loss is 0.792565
-----------------------
-----------------------
Epoch 4 end: 4.693384647369385, F1 is: 0.3333333333333333
V

-----------------------
Train - Epoch 12 - batch 100, batch loss is 0.721375
-----------------------
-----------------------
Epoch 12 end: 5.180992126464844, F1 is: 0.3869933022181284
Validation loss: 15.9895592 - F1 score: 0.4475066
0 class -> precision: 0.2358382 - recall: 0.6800000
1 class -> precision: 0.8251366 - recall: 0.4066427
precision: 0.5304874 - recall: 0.5433214 - MACRO
-----------------------
-----------------------
Train - Epoch 13 - batch 100, batch loss is 0.719126
-----------------------
-----------------------
Epoch 13 end: 5.775055170059204, F1 is: 0.39845021292279187
Validation loss: 15.9520283 - F1 score: 0.4830097
0 class -> precision: 0.2477184 - recall: 0.6333333
1 class -> precision: 0.8299845 - recall: 0.4820467
precision: 0.5388515 - recall: 0.5576900 - MACRO
-----------------------
-----------------------
Train - Epoch 14 - batch 100, batch loss is 0.717125
-----------------------
-----------------------
Epoch 14 end: 6.190668344497681, F1 is: 0.4093237803

-----------------------
Train - Epoch 32 - batch 100, batch loss is 0.699528
-----------------------
-----------------------
Epoch 32 end: 5.79121994972229, F1 is: 0.543954674698049
Validation loss: 15.5549717 - F1 score: 0.5029488
0 class -> precision: 0.2640000 - recall: 0.1100000
1 class -> precision: 0.7928627 - recall: 0.9174147
precision: 0.5284313 - recall: 0.5137074 - MACRO
-----------------------
-----------------------
Train - Epoch 33 - batch 100, batch loss is 0.699082
-----------------------
-----------------------
Epoch 33 end: 6.175074338912964, F1 is: 0.5465176099496428
Validation loss: 15.5436182 - F1 score: 0.5065737
0 class -> precision: 0.2764228 - recall: 0.1133333
1 class -> precision: 0.7939582 - recall: 0.9201077
precision: 0.5351905 - recall: 0.5167205 - MACRO
-----------------------
-----------------------
Train - Epoch 34 - batch 100, batch loss is 0.698586
-----------------------
-----------------------
Epoch 34 end: 6.066689729690552, F1 is: 0.5520621355892

-----------------------
Train - Epoch 52 - batch 100, batch loss is 0.692263
-----------------------
-----------------------
Epoch 52 end: 6.33958625793457, F1 is: 0.6083694097262157
Validation loss: 15.4235916 - F1 score: 0.5024631
0 class -> precision: 0.2830189 - recall: 0.1000000
1 class -> precision: 0.7935780 - recall: 0.9317774
precision: 0.5382984 - recall: 0.5158887 - MACRO
-----------------------
-----------------------
Train - Epoch 53 - batch 100, batch loss is 0.692036
-----------------------
-----------------------
Epoch 53 end: 5.540123701095581, F1 is: 0.6107449564392418
Validation loss: 15.4207344 - F1 score: 0.5020455
0 class -> precision: 0.2803738 - recall: 0.1000000
1 class -> precision: 0.7934200 - recall: 0.9308797
precision: 0.5368969 - recall: 0.5154399 - MACRO
-----------------------
-----------------------
Train - Epoch 54 - batch 100, batch loss is 0.691843
-----------------------
-----------------------
Epoch 54 end: 5.656908988952637, F1 is: 0.612784938827

-----------------------
Train - Epoch 72 - batch 100, batch loss is 0.682256
-----------------------
-----------------------
Epoch 72 end: 5.64538311958313, F1 is: 0.7148650684122895
Validation loss: 15.5349445 - F1 score: 0.5455074
0 class -> precision: 0.2983193 - recall: 0.2366667
1 class -> precision: 0.8052721 - recall: 0.8500898
precision: 0.5517957 - recall: 0.5433782 - MACRO
-----------------------
-----------------------
Train - Epoch 73 - batch 100, batch loss is 0.682390
-----------------------
-----------------------
Epoch 73 end: 5.847628116607666, F1 is: 0.7158800800414921
Validation loss: 15.5377827 - F1 score: 0.5475674
0 class -> precision: 0.3004115 - recall: 0.2433333
1 class -> precision: 0.8061486 - recall: 0.8473968
precision: 0.5532801 - recall: 0.5453651 - MACRO
-----------------------
-----------------------
Train - Epoch 74 - batch 100, batch loss is 0.682510
-----------------------
-----------------------
Epoch 74 end: 5.6783130168914795, F1 is: 0.71700274571

In [80]:
gc.collect()

522